# Task 5

In [ ]:
import os
import torch
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from rouge_score import rouge_scorer
from tqdm import tqdm
from transformers import logging as transformers_logging

In [ ]:
# Set seed for reproducibility
torch.manual_seed(42)

## 5.1 Preprocessing

In [ ]:
# Load and preprocess SQuAD dataset

dataset = load_dataset("squad")

print("Number of training examples:", len(dataset['train']))
print("Number of validation examples:", len(dataset['validation']))

dataset['train'][0]

In [ ]:
dataset['train'][1]

In [ ]:
# Data Filtering
filtered_dataset = dataset.filter(lambda x: x["answers"]['text'][0].upper() !="CANNOTANSWER")
print("Size of training set after removing unanswerable questions:", len(filtered_dataset['train']))
print("Size of validation set after removing unanswerable questions:", len(filtered_dataset['validation']))

In [ ]:
# Creating a Testing Set

# Take subsets to avoid overload
train_dataset = dataset["train"].select(range(10000,11000))
val_dataset = dataset["validation"].select(range(3000,3100))
test_dataset = dataset["validation"].select(range(3100, 3200))  # No official SQuAD test set

training_set = train_dataset
validation_set = val_dataset
testing_set = test_dataset

print("Size of training set:", len(train_dataset))
print("Size of validation set:", len(val_dataset))
print("Size of testing set:", len(test_dataset))

In [ ]:
# Loading the Tokenizer
MODEL_NAME = "t5-small"

MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
def encode_question_and_context(question, context):
    return f"question: {question}  context: {context}"

# Obtains the context, question and answer from a given sample.
def extract_sample_parts(sample):
    context = sample["context"]
    question = sample["question"]
    answer = sample["answers"]['text'][0]
    question_with_context = encode_question_and_context(question, context)
    return (question_with_context, question, answer)

# Encodes the sample, returning token IDs.
def preprocess(sample):
    # Extract data from sample.
    question_with_context, question, answer = extract_sample_parts(sample)

    # Using truncation causes the tokenizer to emit a warning for every sample.
    # This will generate a significant amount of messages, and likely crash
    # your browser tab. We temporarily disable log messages to work around this.
    # See https://github.com/huggingface/transformers/issues/14285
    old_level = transformers_logging.get_verbosity()
    transformers_logging.set_verbosity_error()
    
    # Generate tokens for the input.
    # We include both the context and the question (first two parameters).
    input_tokens = tokenizer(question_with_context, question, padding="max_length",
                             truncation=True, max_length=MAX_INPUT_LENGTH)

    # Generate tokens for the expected answer. There is no need to include the 
    output_tokens = tokenizer(answer, padding="max_length", truncation=True,
                              max_length=MAX_OUTPUT_LENGTH)

    # Restore old logging level, see above.
    transformers_logging.set_verbosity(old_level)

    # The output of the tokenizer is a map containing {input_ids, attention_mask}.
    # For trianing, we need to add the labels (answer/output tokens) to the map.
    input_tokens["labels"] = np.array(output_tokens["input_ids"])

    return input_tokens

In [ ]:
# Preprocess the datasets
training_set_enc = train_dataset.map(preprocess, batched=False)
validation_set_enc = val_dataset.map(preprocess, batched=False)
testing_set_enc = test_dataset.map(preprocess, batched=False)

In [ ]:
# Prepare 20 data points for qualitative analysis
q_data = test_dataset.select(range(20))
q_data

## 5.2 Fine-tuning the T5 Model

In [ ]:
# Loading the Model
# Ensure the resources for any existing model have been freed.
try:
    del model
except NameError:
    pass
    
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
columns = ["input_ids", "attention_mask", "labels"]
training_set_enc.set_format(type="torch", columns=columns)
validation_set_enc.set_format(type="torch", columns=columns)
testing_set_enc.set_format(type="torch", columns=columns)

### First Setting

In [ ]:
from transformers import EarlyStoppingCallback

# hyperparameter: setting1

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3, #5-10
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4, 
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    metric_for_best_model = "loss", 
    load_best_model_at_end = True #Early Stopping
)

# Train T5 model: setting1

model.train()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_set_enc,
    eval_dataset=validation_set_enc,
    processing_class=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer),
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)], #Early Stopping
)

trainer.train()

### Second Setting

In [ ]:
try:
    del model
except NameError:
    pass

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
# hyperparameter: setting2

training_args_change = TrainingArguments(
    output_dir="./results",
    num_train_epochs=8, 
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=20,
    metric_for_best_model = "loss", 
    load_best_model_at_end = True #Early Stopping
)

# Train T5 model: setting2

model.train()
trainer_change = Trainer(
    model=model,
    args=training_args_change,
    train_dataset=training_set_enc,
    eval_dataset=validation_set_enc,
    processing_class=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer),
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)], #Early Stopping
)

trainer_change.train()

In [ ]:
# Saving the Model
trainer.save_model("t5_pretrained")

## 5.3 Evaluation the fine-tuned model

#### Evaluation Functions

- display_evaluation(setname, results)
- generate_response(tokenizer, model, question)
- generate_answers(tokenizer, model, dataset, use_context=True, limit=None)
- display_answer_and_references(question, answer, reference)
- compute_average_score(scores, metric, key)
- compute_rouge(predictions, references)

In [ ]:
from itertools import batched
def display_evaluation(setname, results):
    print(f"{setname} Set Loss:", round(results["eval_loss"], 3))

# Generates a response for a single input/question.
def generate_response(tokenizer, model, question):
    # Convert the sentences into a list of numeric tokens. We instruct the tokenizer
    # to return PyTorch tensors ("pt") so that we can feed them directly into the model.
    tokenized = tokenizer(question, return_tensors="pt", padding=True, truncation=True,
                          max_length=MAX_OUTPUT_LENGTH).to(model.device)
    # Generate outputs using the model.
    with torch.no_grad():
        outputs = model.generate(**tokenized)
        
    # The model outputs a list of numeric tokens. To convert these tokens back to
    # sentences, we can use the batch_decode function from the tokenizer.
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs
    
# Generates a list of responses from the specified model, optionally including
# the context in the prompt. If limit is set, then answers will only be generated
# for the first N questions of the dataset.
def generate_answers(tokenizer, model, dataset, use_context=True, limit=None):
    # Subsampling if requested.
    if limit is not None:
        dataset = dataset.select(range(limit))
        
    # Create list of encoded tokens, similarly to how we preprocessed the data for
    # training. We do this so we can use batch processing to speed up inference.
    questions = []
    inputs = []
    references = []
    for sample in dataset:
        question_with_context, question, answer = extract_sample_parts(sample)
        
        # Only include the context if the caller requested it.
        if use_context:
            inputs.append(question_with_context)
        else:
            inputs.append(question)
            
        # Include the original question/answer.
        questions.append(question)
        references.append(answer)
        
    # Generate responses for each of the prompts/inputs.
    # Submitting each question to the model separately would significantly
    # increase processing time, especially if the model is located on the GPU.
    # Instead, we group questions together in the same batch size that we used
    # for training.
    outputs = []
    for samples in batched(inputs, 128):
        # Python's batched() function returns a tuple of the batch
        # size, which we have to first convert to a list.
        responses = generate_response(tokenizer, model, list(samples))
        
    # generate_responses() returns an equal-sized list of responses.
    outputs.extend(responses)
    
    # The length of the reference responses should equal the length of the
    # generated responses.
    assert (len(outputs) == len(references))
    return outputs, references, questions

def display_answer_and_references(question, answer, reference):
    print("Question:", question)
    print("Generated answer:", answer)
    print("Reference answer:", reference)
    print()


#### ROUGE

In [ ]:
# Computes the average score of a given metric from a list of ROUGE scores.
def compute_average_score(scores, metric, key):
    total = 0
    for i in range(len(scores)):
        # Since it's not a map, we have to manually read the attribute.
        total += getattr(scores[i][metric], key)
    return total / len(scores)
    
# Computes ROGUE-1, ROGUE-2 and ROGUE-L scores for the given generated
# answers and reference answers.
def compute_rouge(predictions, references):
    # Compute ROUGE-1, ROGUE-2 and ROUGE-L.
    metrics = ["rouge1", "rouge2", "rougeL"]
    
    # Use Porter stemmer to strip word suffixes to improve matching.
    scorer = rouge_scorer.RougeScorer(metrics, use_stemmer=True)
    
    # For each answer/reference pair, compute the ROUGE metrics.
    scores = []
    for prediction, reference in zip(predictions, references):
        scores.append(scorer.score(reference, prediction))
        
    # Compute the average precision, recall and F1 score for each metric.
    results = {}
    for metric in metrics:
        for k in ["precision", "recall", "fmeasure"]:
            results[f"{metric}_{k}"] = compute_average_score(
                scores, metric, k)
    return results

#### ROUGE Metrics: Fine-tuned model

In [ ]:
# Switch the model to evaluation mode, disabling dropout etc layers.
model.eval()

# Evaluate the datasets.
display_evaluation("Training", trainer.evaluate(training_set_enc))
display_evaluation("Testing", trainer.evaluate(testing_set_enc))

In [ ]:
answers_ctx, refs_ctx, questions_ctx = generate_answers(tokenizer, model, testing_set, True, 100)
answers_noctx, refs_noctx, questions_noctx = generate_answers(tokenizer, model, testing_set, False, 100)

In [ ]:
print("ROUGE with context:", compute_rouge(answers_ctx, refs_ctx))
print()
print("ROUGE without context:", compute_rouge(answers_noctx, refs_noctx))

## 5.4 Generative Avalysis

In [ ]:
# 5.4 Generative Analysis
def display_answer_and_references(question, answer, reference):
    print("Question:", question)
    print("Generated answer:", answer)
    print("Reference answer:", reference)
    print()

# 5.4.a question + context
print("*** With context ***")
for i in range(5):
    display_answer_and_references(questions_ctx[i], answers_ctx[i],
                                  refs_ctx[i])

# 5.4.b question
print("*** Without context ***")
for i in range(5):
    display_answer_and_references(questions_noctx[i],
                                  answers_noctx[i], refs_noctx[i])

## 5.5 Comparison with a Pre-trained model

In [ ]:
# Load and preprocess SQuAD dataset
dataset = load_dataset("squad")

In [ ]:
# Take subsets to avoid overload
train_dataset = dataset["train"].select(range(10000,11000))
val_dataset = dataset["validation"].select(range(3000,3100))
test_dataset = dataset["validation"].select(range(3100, 3200))  # No official SQuAD test set

training_set = train_dataset
validation_set = val_dataset
testing_set = test_dataset

print("Size of training set:", len(train_dataset))
print("Size of validation set:", len(val_dataset))
print("Size of testing set:", len(test_dataset))

In [ ]:
MODEL_NAME = "mrm8488/t5-base-finetuned-squadv2"

MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128

In [ ]:
try:
    del model
except NameError:
    pass

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
# Evaluation
# Switch the model to evaluation mode, disabling dropout etc layers.
model.eval()

In [ ]:
answers_ctx, refs_ctx, questions_ctx = generate_answers(
    tokenizer, model, testing_set, True, 100)
answers_noctx, refs_noctx, questions_noctx = generate_answers(
    tokenizer, model, testing_set, False, 100)

In [ ]:
print("ROUGE with context:", compute_rouge(answers_ctx, refs_ctx))
print()
print("ROUGE without context:", compute_rouge(answers_noctx, refs_noctx))

In [ ]:
def display_answer_and_references(question, answer, reference):
    print("Question:", question)
    print("Generated answer:", answer)
    print("Reference answer:", reference)
    print()
      
print("*** With context ***")
for i in range(5):
    display_answer_and_references(questions_ctx[i], answers_ctx[i],
                                  refs_ctx[i])

# 5.4.b question
print("*** Without context ***")
for i in range(5):
    display_answer_and_references(questions_noctx[i],
                                  answers_noctx[i], refs_noctx[i])